## Resting State Complexity is Associated with Oral Ketamine Treatment Response: A Bayesian Analysis of Lempel-Ziv Complexity and Multi-Scale Entropy 
Authors: Mitchell J. S., Anijärv, T.E., Can, A., Dutton, M., Hermens, D.F., & Lagopoulos, J.

Code created by Toomas Erik Anijärv and Jules Mitchell January 2024.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [1]:
# Import packages
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import neurokit2 as nk
from scipy.io import loadmat

# Set and confirm default directory
os.chdir('')
os.getcwd()
mne.set_log_level('error')

# Import functions
import signal_processing.pre_process as pre_process
import basic.arrange_data as arrange

In [ ]:
# Set relevant folder paths
raw_folder = 'Data\Raw' # Folder where to get the raw EEG files
# raw_folder = 'studies\OKTOS'

# Folder where to export the clean epochs files
clean_folder = 'Data\Clean'
#clean_folder = 'studies\OKTOS\derivatives'

# Folder to save the results
results_folder = 'Data\Results'
#results_folder = 'studies\OKTOS\derivatives\analysis'

# Define timepoint sub-folders (i.e. edit if you don't want to run all timepoint)
exp_folder = ['BAS', 'POST', 'FUP']

### EEGLAB file epoching

In [ ]:
# Define EEG file information
n_channels = 32
sampling_freq = 250  # in Hertz
info = mne.create_info(n_channels, sfreq=sampling_freq)

# Loop over timepoint folders 
for exp in exp_folder:

    # Get directories of raw EEG files and set export directory for clean files
    dir_inprogress = os.path.join(raw_folder,exp)
    print(dir_inprogress)
    export_dir = os.path.join(clean_folder,exp)
    file_dirs, subject_names = arrange.read_files(dir_inprogress,'.set')

    # Loop through all the subjects' directories (EEG files directories)
    for i in range(len(file_dirs)):
        # Read in the raw EEG data
        #data = loadmat(file_dirs[i])
        #raw = mne.io.RawArray(file_dirs[i], info, first_samp=0, copy='auto', verbose=None)
        raw = mne.io.read_raw_eeglab(file_dirs[i], eog=(), preload=False, uint16_codec=None, montage_units='mm', verbose=None)

        epochs = mne.make_fixed_length_epochs(raw, duration=5.0, preload=False, reject_by_annotation=True, proj=True, overlap=0.0, id=1, verbose=None)

        # Save epoched data
        mne.Epochs.save(epochs, fname='{}/{}_clean-epo.fif'.format(export_dir,
                                                                    subject_names[i]),
                                                                    overwrite=True)

### Complexity Calculation

In [ ]:
# Define arguments for each metric (LZC args provided if you wish to calculate permutation LZC also)
lzc_args = dict(delay=1, dimension = 3, permutation = True)
mse_args = dict(method="MSEn", scale=20, dimension=2, tolerance='sd')

# Define channels
channels = [['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4',
			 'P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']]

In [ ]:
# Loop through all experiments (i.e., timepoints)
df = pd.DataFrame()
for exp in exp_folder:
    # Get directories of clean EEG files and set export directory
    dir_inprogress = os.path.join(clean_folder,exp)
    file_dirs, subject_names = arrange.read_files(dir_inprogress,'_clean-epo.fif')
    print(subject_names)

    # Doing it this way (based on the length of subject names) ensures the correct values are associated with each file
    IDs = []
    for s in subject_names:
	    IDs.append(s[0:6])
     
    Tx = []
    for s in subject_names:
        Tx.append(s[7:13])
    
    Task = []
    for s in subject_names:
        Task.append(s[14:16])

    # Create dictionary to access response status based on ID
    sample = [
    'sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05',
    'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10',
    'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15',
    'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20',
    'sub-21', 'sub-22', 'sub-23', 'sub-24', 'sub-25',
    'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-31',
    'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36',
    'sub-38', 'sub-39', 'sub-40']

    resp_status = [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]

    # Create a dictionary with participant IDs as keys and response statuses as values
    response_dict = dict(zip(sample, resp_status))
	
    # Loop through all the subjects' directories (EEG files directories)
    df_exp = pd.DataFrame(index=range(len(file_dirs)))
    df_exp_channels = pd.DataFrame(index=range(len(file_dirs)), columns = channels)
    df_exp_channels_perm = pd.DataFrame(index=range(len(file_dirs)), columns = channels)

    for i in range(len(file_dirs)):
        # Update df_exp and df_exp_channels with timepoint (Tx) and Task (e.g. Eyes closed(EC))
        df_exp.loc[i, 'Subject'] = IDs[i]
        df_exp.loc[i, 'Timepoint'] = Tx[i]
        df_exp.loc[i, 'Task'] = Task[i]
        df_exp.loc[i, 'Responder'] = response_dict[IDs[i]]
        
        df_exp_channels.loc[i, 'Subject'] = IDs[i]
        df_exp_channels.loc[i, 'Timepoint'] = Tx[i]
        df_exp_channels.loc[i, 'Task'] = Task[i]
        df_exp_channels.loc[i, 'Responder'] = response_dict[IDs[i]]

        df_exp_channels_perm.loc[i, 'Subject'] = IDs[i]
        df_exp_channels_perm.loc[i, 'Timepoint'] = Tx[i]
        df_exp_channels_perm.loc[i, 'Task'] = Task[i]
        df_exp_channels_perm.loc[i, 'Responder'] = response_dict[IDs[i]]

        # Read the clean data from the disk
        print('{}: {} ({}/{})'.format(exp, subject_names[i], i+1, len(file_dirs)))
        epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]),
                                                                    verbose=False)
        
        # Convert data file to dataframe 
        df_epochs = epochs.to_data_frame()
        
        ### Lempel-Ziv complexity
        # Go through all the channels signals
        lzc_i = []
        lzc_med_i = []
        lzc_perm = []
        for ch in epochs.info['ch_names']:
            print(ch)
            # Go through all epochs in the current channel signal
            lzc_ch = []
            lzc_ch_med = []
            lzc_ch_perm = []
            for epo in df_epochs['epoch'].unique():
                # Calculate Lempel-Ziv Complexity (LZC) for the current epoch
                epo_signal = df_epochs[df_epochs['epoch']==epo][ch]
                # Normal (mean)
                lzc_epo, info_1 = nk.complexity_lempelziv(epo_signal, symbolize='mean', permutation = False)
                lzc_ch.append(lzc_epo)
                # Normal (median)
                lzc_epo_med, info_2 = nk.complexity_lempelziv(epo_signal, symbolize='median', permutation = False)
                lzc_ch_med.append(lzc_epo)
                # Permutation
                lzc_epo_perm, info_3 = nk.complexity_lempelziv(epo_signal, **lzc_args)
                lzc_ch_perm.append(lzc_epo_perm)
                
            # Average all epochs' LZC values to get a single value for the channel & add to list
            lzc_i.append(np.mean(lzc_ch))
            lzc_med_i.append(np.mean(lzc_ch_med))
            lzc_perm.append(np.mean(lzc_ch_perm))

            # Add channel complexity values to dataframe
            df_exp_channels.loc[i, ch] = np.mean(lzc_ch)
            df_exp_channels_perm.loc[i, ch] = np.mean(lzc_ch_perm)

        # Average all the channels' LZC values to get a single value for the subject & add to master dataframe
        lzc_i_mean = np.mean(lzc_i)
        lzc_med_i_mean = np.mean(lzc_med_i)
        lzc_perm_mean = np.mean(lzc_perm)
        df_exp.loc[i, 'LZC'] = lzc_i_mean
        df_exp.loc[i, 'LZC_M'] = lzc_med_i_mean
        df_exp.loc[i, 'LZC_P'] = lzc_perm_mean

        ### Multiscale Sample Entropy
        # Go through all the channels signals
        mse_i = []
        mse_vals_i = np.zeros(shape=(len(epochs.info['ch_names']), mse_args['scale']))

        for c, ch in enumerate(epochs.info['ch_names']):
            # Go through all epochs in the current channel signal
            mse_ch = []
            mse_vals_epo = []

            for epo in df_epochs['epoch'].unique():
                # Calculate Multiscale Sample Entropy (MSE) measures for the current epoch
                epo_signal = df_epochs[df_epochs['epoch']==epo][ch]
                mse_epo, info_3 = nk.entropy_multiscale(epo_signal.to_numpy(), **mse_args)

                # Get the total and scales' MSE values for the current epoch & add to list including all epochs
                mse_ch.append(mse_epo)
                mse_vals_epo.append(info_3.get('Value'))

            # Average all epochs' MSE values for every channel for the subject
            mse_vals_i[c] = np.mean(mse_vals_epo, axis=0)

            # Average all epochs' MSE totals to get a single value for the channel & add to list
            mse_i.append(np.mean(mse_ch))

        # Average all the channels' MSE and MSPLZC totals & values to get global value
        mse_i_mean = np.mean(mse_i)
        mse_vals_i_mean = np.mean(mse_vals_i, axis=0)
        
        # Add total MSE to dataframe for the subject
        df_exp.loc[i, 'MSE (total)'] = mse_i_mean

        # Add all scales' MSE values to dataframe for the subject
        for scl in range(mse_args['scale']):
            df_exp.loc[i, 'MSE (scale={})'.format(scl+1)] = mse_vals_i_mean[scl]
        
    # Add the current timepoint data to the master dataframe
    df = pd.concat([df, df_exp])

df.to_excel('{}/Complexity_results.xlsx'.format(results_folder))
df_exp_channels.to_excel('{}/LZC_chan_results.xlsx'.format(results_folder))
df_exp_channels_perm.to_excel('{}/LZC_perm_chan_results.xlsx'.format(results_folder))